Sort people into leagues for curling
KL 8/30/2016

#This version starts with a random league and puts a random subset of people into each league;
updated to use new website export format

#REMEMBER: this will skip league managers for the moment

#something wrong with this in the iterations after first choices

In [31]:
if True:
    %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [32]:
setSeed = False

In [33]:
#set the random seed...if a number...will be able to recreate the random numbers
if setSeed == True:
    SEED = 42
elif setSeed == False:
    SEED = None

In [34]:
import pandas as pd
import numpy as np
import random
import math
import csv
import os.path

#used this to step into the function and debug it, also need line with Tracer()() 
from IPython.core.debugger import Tracer 

Define some functions up top

In [35]:
def addOne(leagueDict,oneLeague,oneName):
    if len(leagueDict[oneLeague])==0: #nothing there yet
        leagueDict[oneLeague] = oneName
    else:     #need to append
        leagueDict[oneLeague] = leagueDict[oneLeague].append(oneName)
    return leagueDict

In [36]:
def addManager(leagueDictionary,r,manager): #here r is the index in...
    useRow = r.index[0]
    oneName = regSorted.loc[useRow,['lastname', 'firstname']] 
    oneName = oneName.to_frame().transpose() #need this format to make nice output
    leagueDictionary[manager] = oneName
    regSorted.loc[useRow,manager]=np.nan
    regTracking.loc[useRow,manager]='yes'
    return leagueDictionary

Read in the data

In [37]:
#updated 8/30/2016 to use file from website
fName = 'BackupList_Registrations.csv' 
importInfo=pd.read_csv(fName)

In [38]:
#shuffle the information from the website site to match the format in the rest of this file
openLeagues = ['Sunday Night', 'Monday 4PM', 'Monday Night',
               'Tuesday 4PM','Wednesday 4PM','Thursday 4PM'] #updated to match the priority list
otherLeagues = ['League_4_Men', 'League_7_Ladies' ,'League_Scrod']
priorities = ['League_Pri_1','League_Pri_2','League_Pri_3','League_Pri_4','League_Pri_5','League_Pri_6']

regInfo = importInfo.loc[:,(['firstname','lastname','email','League_Number_Open'] + otherLeagues)]

for item in openLeagues:
    regInfo[item] = np.nan

nPeople = len(regInfo.index)
for idx in range(0,nPeople):
    for idxP in enumerate(priorities):
        #remember: enumerate will allow idxP[0] is the number, while idxP[1] is the value
        getOne = importInfo.loc[idx,idxP[1]]
        if getOne != 'None':
            regInfo.loc[idx,getOne] = idxP[0] + 1  #want 1,2,3 and not python zero index

In [39]:
#first, how many random numbers so I need?
nPeople = len(regInfo.index)
np.random.seed(SEED)
regInfo['randomNumCol'] = np.random.choice(range(nPeople), nPeople,replace = False)
##so now, moving back to the list of people, sort the list by the random number
#...still need to randomize people bc might have case where I can't grant first choice
regSorted = regInfo.sort_values(by = 'randomNumCol', ascending=True)
regSorted = regSorted.set_index(['randomNumCol'])

#copy so I can have an output tracking what was done
regTracking = regSorted.copy(deep=True)

setup the places to store the output

In [40]:
random.seed(SEED)
randLeagues = list(openLeagues) #make a copy of the list
random.shuffle(randLeagues) #does this in place, so don't make a new variable
randLeagues

['Tuesday 4PM',
 'Monday 4PM',
 'Wednesday 4PM',
 'Monday Night',
 'Sunday Night',
 'Thursday 4PM']

In [41]:
#might not want to randomize the leagues...changes outcome depending on how people prioritize

In [42]:
#want a way to quantify the most popular leagues...go back to regInfo
countFirst = {}
for name in openLeagues:
    #countFirst[name] = pd.DataFrame()
    countFirst[name] = len(regInfo.loc[regInfo[name]==1])

countFirst

#based on this...and the multiple runs, maybe should start with the least popular leagues in order
#to get the most people into their first choice
#though if we run Thursday 4 pm, Sunday, and Mondy 4 pm, have 1 person who doesn't even get 3rd choice

{'Monday 4PM': 5,
 'Monday Night': 15,
 'Sunday Night': 12,
 'Thursday 4PM': 15,
 'Tuesday 4PM': 7,
 'Wednesday 4PM': 7}

In [43]:
#setup the dictionary to hold the people information (start with openLeagues only)
forDictionary = openLeagues
#forDictionary.extend(otherLeagues)
leagueDict_open = {}
for name in forDictionary:
    leagueDict_open[name] = pd.DataFrame()

In [44]:
regSorted.head(2)

,firstname,lastname,email,League_Number_Open,League_4_Men,League_7_Ladies,League_Scrod,Sunday Night,Monday 4PM,Monday Night,Tuesday 4PM,Wednesday 4PM,Thursday 4PM
randomNumCol,,,,,,,,,,,,,
0,Mary,Kavanagh,M84PAT@aol.com,2,No,No,No,1.0,NaN,NaN,4.0,2.0,3.0
1,Joseph,Tamucci,jtamucci@comcast.net,3,Yes,No,No,NaN,2.0,NaN,4.0,3.0,1.0


In [45]:
if False:
    #put the league managers into their league and set their the choice for that league to NaN
    manager = 'Sunday Night'
    r = regSorted.loc[(regSorted['lastname']=='Fragaszy') & (regSorted['firstname']=='Bill')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Monday 4PM'
    r = regSorted.loc[(regSorted['lastname']=='Brown') & (regSorted['firstname']=='Peter')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Monday Night'
    r = regSorted.loc[(regSorted['lastname']=='DiMassa') & (regSorted['firstname']=='Diane')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Tuesday 4PM'
    r = regSorted.loc[(regSorted['lastname']=='Thomson') & (regSorted['firstname']=='Kenny')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Wednesday 4PM'
    r = regSorted.loc[(regSorted['lastname']=='Murphy') & (regSorted['firstname']=='Katie')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

    manager = 'Thursday 4PM'
    r = regSorted.loc[(regSorted['lastname']=='Gallagher') & (regSorted['firstname']=='Bill')]
    leagueDict_open = addManager(leagueDict_open,r,manager)

In [46]:
#set the limit for each league:
sevenTeams = 20 #change to 28 (remember league manager already in here)
fourteenTeams = 50

d = {'Sunday Night': sevenTeams,
     'Monday 4PM': sevenTeams,
     'Monday Night': fourteenTeams,
     'Tuesday 4PM': sevenTeams,
     'Wednesday 4PM': sevenTeams,
     'Thursday 4PM': sevenTeams,}

nLimit = pd.DataFrame({'nLimit' : d})
del d

In [47]:
issueNames = ['notInFirst','notInSecond','notInThird','notInFourth','notInFifth','noSecondChoice','noThirdChoice']

#setup the dictionary to hold the issues
trackIssues = {}
for name in issueNames:
    trackIssues[name] = pd.DataFrame()

In [48]:
trackIssues

{'noSecondChoice': Empty DataFrame
 Columns: []
 Index: [], 'noThirdChoice': Empty DataFrame
 Columns: []
 Index: [], 'notInFifth': Empty DataFrame
 Columns: []
 Index: [], 'notInFirst': Empty DataFrame
 Columns: []
 Index: [], 'notInFourth': Empty DataFrame
 Columns: []
 Index: [], 'notInSecond': Empty DataFrame
 Columns: []
 Index: [], 'notInThird': Empty DataFrame
 Columns: []
 Index: []}

In [49]:
pd.options.mode.chained_assignment = None  # default='warn'

In [50]:
for oneLeague in randLeagues:    
    #how many spaces are left in this league?
    nRemaining = nLimit.loc[oneLeague][0] - len(leagueDict_open[oneLeague])
    if nRemaining > 0:      
        #find list of people interested in that league
        subset = regSorted.loc[regSorted[oneLeague]==1]
        subset.reset_index(inplace = True)
        nPeople = len(subset.index)
        subset.loc[:,('subsetRandomNumCol')] = np.random.choice(range(nPeople),nPeople, replace = False)

        for idx in range(0,nPeople): #remember nPeople in this version is only subset for a league
            oneName = subset.loc[idx,['lastname', 'firstname']] 
            oneName = oneName.to_frame().transpose() #need this format to make nice output

            #does this person want another league?
            rt = regTracking.loc[idx,openLeagues]
            cHave = len(rt[rt=='yes'])
            del rt
            
            if cHave < regTracking.loc[idx,'League_Number_Open']: #person wants more 
                if subset.loc[idx,'subsetRandomNumCol'] < (nRemaining): #random # < number of places remaining
                    leagueDict_open = addOne(leagueDict_open,oneLeague,oneName)  
                    $$$ERROR is HERE (should not be idx...)
                    regSorted.loc[idx,oneLeague] = np.nan
                    regTracking.loc[idx,oneLeague] = 'yes'

                else:
                    #no space in first choice league:
                    trackIssues = addOne(trackIssues,'notInFirst',oneName)
                    #set their first choice to 'no' and move onto second choices
                    newI = subset.loc[idx,'randomNumCol'] ###remember: this is the index into regSorted and regTracking
                    regTracking.loc[newI,oneLeague] = 'no'
                    regSorted.loc[newI,oneLeague] = np.nan

                    #use the idx into subset to go back into regTracking
                    newPriority = regSorted.loc[newI,openLeagues].dropna().min()

                    if math.isnan(newPriority) == False: #have another league listed
                        league2 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                        #first need to check if the league has space
                        if len(leagueDict_open[league2]) < (nLimit.loc[league2,'nLimit']):
                            leagueDict_open = addOne(leagueDict_open,league2,oneName)
                            regTracking.loc[newI,league2] = 'yes'
                            regSorted.loc[newI,league2] = np.nan
                        else:
                            #no space in second choice league
                            trackIssues = addOne(trackIssues,'notInSecond',oneName)
                            #set their first choice to 'no' and move onto second choices
                            regTracking.loc[newI,league2] = 'no' 
                            regSorted.loc[newI,league2] = np.nan

                            #use the idx into subset to go back into regTracking
                            priority3 = regSorted.loc[newI,openLeagues].dropna().min()

                            if math.isnan(priority3) == False: #have another league listed
                                league3 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                                #first need to check if the league has space
                                if len(leagueDict_open[league3]) < (nLimit.loc[league3,'nLimit']):
                                    leagueDict_open = addOne(leagueDict_open,league3,oneName)
                                    regTracking.loc[newI,league3] = 'yes'
                                    regSorted.loc[newI,league3] = np.nan
                                else:
                                    print oneName.values.ravel()[1],oneName.values.ravel()[0],': no space in third choice league'
                                    trackIssues = addOne(trackIssues,'notInThird',oneName)
                                    regTracking.loc[newI,league3] = 'no'
                            else: 
                                #no other league listed
                                trackIssues = addOne(trackIssues,'noThirdChoice',oneName)
                    elif math.isnan(newPriority):
                        #print oneName.values.ravel()[1],oneName.values.ravel()[0], ': did not list a second league'
                        trackIssues = addOne(trackIssues,'noSecondChoice',oneName)
        del subset
    elif nRemaining == 0:
        print oneLeague, 'no more space in league'

In [51]:
#go through ...this time start with second choices
#SOMETHING NOT RIGHT...looking for more choices when people should be done...

In [52]:
for oneLeague in randLeagues:    
    #how many spaces are left in this league?
    nRemaining = nLimit.loc[oneLeague][0] - len(leagueDict_open[oneLeague])
    if nRemaining > 0:      
        #find list of people interested in that league
        subset = regSorted.loc[regSorted[oneLeague]==2]
        #only move on if people have actually ranked this league as their second choice...
        #Tracer()()
        if len(subset) > 0:           
            subset.reset_index(inplace = True)
            nPeople = len(subset.index)
            #Tracer()()
            subset.loc[:,('subsetRandomNumCol')] = np.random.choice(range(nPeople),nPeople, replace = False)

            for idx in range(0,nPeople): #remember nPeople in this version is only subset for a league
                oneName = subset.loc[idx,['lastname', 'firstname']] 
                oneName = oneName.to_frame().transpose() #need this format to make nice output

                #does this person want another league?
                rt = regTracking.loc[idx,openLeagues]
                cHave = len(rt[rt=='yes'])
                del rt

                if cHave < regTracking.loc[idx,'League_Number_Open']: #person wants more 
                    if subset.loc[idx,'subsetRandomNumCol'] < (nRemaining): #random # < number of places remaining
                        leagueDict_open = addOne(leagueDict_open,oneLeague,oneName)                
                        regSorted.loc[idx,oneLeague] = np.nan
                        regTracking.loc[idx,oneLeague] = 'yes'

                    else:
                        #no space in first choice league:
                        trackIssues = addOne(trackIssues,'notInSecond',oneName)
                        #set their first choice to 'no' and move onto second choices
                        newI = subset.loc[idx,'randomNumCol'] ###remember: this is the index into regSorted and regTracking
                        regTracking.loc[newI,oneLeague] = 'no'
                        regSorted.loc[newI,oneLeague] = np.nan

                        #use the idx into subset to go back into regTracking
                        newPriority = regSorted.loc[newI,openLeagues].dropna().min()

                        if math.isnan(newPriority) == False: #have another league listed
                            league2 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                            #first need to check if the league has space
                            if len(leagueDict_open[league2]) < (nLimit.loc[league2,'nLimit']):
                                leagueDict_open = addOne(leagueDict_open,league2,oneName)
                                regTracking.loc[newI,league2] = 'yes'
                                regSorted.loc[newI,league2] = np.nan
                            else:
                                #no space in second choice league
                                trackIssues = addOne(trackIssues,'notInThird',oneName)
                                #set their first choice to 'no' and move onto second choices
                                regTracking.loc[newI,league2] = 'no' 
                                regSorted.loc[newI,league2] = np.nan

                                #use the idx into subset to go back into regTracking
                                priority3 = regSorted.loc[newI,openLeagues].dropna().min()

                                if math.isnan(priority3) == False: #have another league listed
                                    league3 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                                    #first need to check if the league has space
                                    if len(leagueDict_open[league3]) < (nLimit.loc[league3,'nLimit']):
                                        leagueDict_open = addOne(leagueDict_open,league3,oneName)
                                        regTracking.loc[newI,league3] = 'yes'
                                        regSorted.loc[newI,league3] = np.nan
                                    else:
                                        print oneName.values.ravel()[1],oneName.values.ravel()[0],': no space in fourth choice league'
                                        trackIssues = addOne(trackIssues,'notInFourth',oneName)
                                        regTracking.loc[newI,league3] = 'no'
                                else: 
                                    #no other league listed
                                    trackIssues = addOne(trackIssues,'noFourthChoice',oneName)
                        elif math.isnan(newPriority):
                            #print oneName.values.ravel()[1],oneName.values.ravel()[0], ': did not list a second league'
                            trackIssues = addOne(trackIssues,'noThirdChoice',oneName)
            del subset
        elif nRemaining == 0:
            print oneLeague, 'no more space in league'

In [53]:
for oneLeague in randLeagues:    
    nRemaining = nLimit.loc[oneLeague][0] - len(leagueDict_open[oneLeague])
    if nRemaining > 0:      
        subset = regSorted.loc[regSorted[oneLeague]==3]
        #only move on if people have actually ranked this league as their second choice...
        #Tracer()()
        if len(subset) > 0:           
            subset.reset_index(inplace = True)
            nPeople = len(subset.index)
            subset.loc[:,('subsetRandomNumCol')] = np.random.choice(range(nPeople),nPeople, replace = False)

            for idx in range(0,nPeople): #remember nPeople in this version is only subset for a league
                oneName = subset.loc[idx,['lastname', 'firstname']] 
                oneName = oneName.to_frame().transpose() #need this format to make nice output

                #does this person want another league?
                rt = regTracking.loc[idx,openLeagues]
                cHave = len(rt[rt=='yes'])
                del rt

                if cHave < regTracking.loc[idx,'League_Number_Open']: #person wants more 
                    if subset.loc[idx,'subsetRandomNumCol'] < (nRemaining): #random # < number of places remaining
                        leagueDict_open = addOne(leagueDict_open,oneLeague,oneName)                
                        regSorted.loc[idx,oneLeague] = np.nan
                        regTracking.loc[idx,oneLeague] = 'yes'

                    else:
                        #no space in first choice league:
                        trackIssues = addOne(trackIssues,'notInThird',oneName)
                        #set their first choice to 'no' and move onto second choices
                        newI = subset.loc[idx,'randomNumCol'] ###remember: this is the index into regSorted and regTracking
                        regTracking.loc[newI,oneLeague] = 'no'
                        regSorted.loc[newI,oneLeague] = np.nan

                        #use the idx into subset to go back into regTracking
                        newPriority = regSorted.loc[newI,openLeagues].dropna().min()

                        if math.isnan(newPriority) == False: #have another league listed
                            league2 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                            #first need to check if the league has space
                            if len(leagueDict_open[league2]) < (nLimit.loc[league2,'nLimit']):
                                leagueDict_open = addOne(leagueDict_open,league2,oneName)
                                regTracking.loc[newI,league2] = 'yes'
                                regSorted.loc[newI,league2] = np.nan
                            else:
                                #no space in second choice league
                                trackIssues = addOne(trackIssues,'notInFourth',oneName)
                                #set their first choice to 'no' and move onto second choices
                                regTracking.loc[newI,league2] = 'no' 
                                regSorted.loc[newI,league2] = np.nan

                                #use the idx into subset to go back into regTracking
                                priority3 = regSorted.loc[newI,openLeagues].dropna().min()

                                if math.isnan(priority3) == False: #have another league listed
                                    league3 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                                    #first need to check if the league has space
                                    if len(leagueDict_open[league3]) < (nLimit.loc[league3,'nLimit']):
                                        leagueDict_open = addOne(leagueDict_open,league3,oneName)
                                        regTracking.loc[newI,league3] = 'yes'
                                        regSorted.loc[newI,league3] = np.nan
                                    else:
                                        print oneName.values.ravel()[1],oneName.values.ravel()[0],': no space in fifth choice league'
                                        trackIssues = addOne(trackIssues,'notInFifth',oneName)
                                        regTracking.loc[newI,league3] = 'no'
                                else: 
                                    #no other league listed
                                    trackIssues = addOne(trackIssues,'noFifthChoice',oneName)
                        elif math.isnan(newPriority):
                            #print oneName.values.ravel()[1],oneName.values.ravel()[0], ': did not list a second league'
                            trackIssues = addOne(trackIssues,'noFourthChoice',oneName)
            del subset
        elif nRemaining == 0:
            print oneLeague, 'no more space in league'

Anne Perella : no space in fifth choice league


In [54]:
#only need to run one more time if have people who want to curl in 4 open leagues

In [55]:
#make a new dictionary for other leagues...concat with openLeagues when done
# do doubles and competitive league by hand...
otherLeagues = ['League_4_Men', 'League_7_Ladies' ,'League_Scrod']

leagueDict_other = {}
for name in otherLeagues:
    leagueDict_other[name] = pd.DataFrame()

In [56]:
if False:
    #put league managers in for other
    manager = 'League_4_Men'
    r = regSorted.loc[(regSorted['lastname']=='Pijanowski') & (regSorted['firstname']=='Brian')]
    leagueDict_other = addManager(leagueDict_other,r,manager)

    manager = 'League_7_Ladies'
    r = regSorted.loc[(regSorted['lastname']=='Galligan') & (regSorted['firstname']=='Natalie')]
    leagueDict_other = addManager(leagueDict_other,r,manager)

    manager = 'League_Scrod'
    r = regSorted.loc[(regSorted['lastname']=='Pilotte') & (regSorted['firstname']=='Courtney Lee')]
    leagueDict_other = addManager(leagueDict_other,r,manager)

In [57]:
#only have to go through once
for idx in range(0,nPeople):
    for oL in otherLeagues:    
        oneName = regSorted.loc[idx,['lastname', 'firstname']] 
        oneName = oneName.to_frame().transpose() #need this format to make nice output
        if regSorted.loc[idx,oL]=='Yes': #careful with trailing spaces
            leagueDict_other = addOne(leagueDict_other,oL,oneName)
            #now that I have added the person to a league, change their priority to NaN
            regTracking.loc[idx,oL] = 'yes'
            regSorted.loc[idx,oL] = np.nan

In [58]:
#combine the two dictionaries
leagueDict_all = leagueDict_open.copy()
leagueDict_all.update(leagueDict_other)

#combine the list of names
allLeagues = list(openLeagues)
allLeagues.extend(otherLeagues)

In [59]:
#export the results to a series of CSV files (CAREFUL...will delete existing files)
for fn in allLeagues:
    fName = '%s.csv' % fn
    #check if the file exists...delete if it does
    if os.path.isfile(fName):
        os.remove(fName)
        
    leagueDict_all[fn].to_csv(fName)

In [60]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

notInFirst :  0  people in this category
notInSecond :  2  people in this category
notInThird :  5  people in this category
notInFourth :  3  people in this category
notInFifth :  1  people in this category
noSecondChoice :  0  people in this category
noThirdChoice :  0  people in this category


['Tuesday 4PM',
 'Monday 4PM',
 'Wednesday 4PM',
 'Monday Night',
 'Sunday Night',
 'Thursday 4PM']

In [272]:
regTracking.to_csv('testing.csv')

In [273]:
regSorted.to_csv('testing2.csv')

In [274]:
regInfo.to_csv('testing3.csv')

In [ ]:
#ran multiple times...order the leagues are randomized will change the output

In [ ]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

In [ ]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

In [ ]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

In [ ]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'

randLeagues